In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf

# Creating classifier model using CNN

## loading the dataset

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale=1.0/255, zoom_range=0.2)
test_generator = ImageDataGenerator(rescale=1.0/255, zoom_range=0.2)

In [3]:
# mask - 0
# without mask - 1

train = train_generator.flow_from_directory(
    'train',
    target_size = (150, 150),
    batch_size = 16,
    class_mode='binary'
)

Found 1314 images belonging to 2 classes.


In [4]:

test = train_generator.flow_from_directory(
    'test',
    target_size = (150, 150),
    batch_size = 16,
    class_mode='binary'
)

Found 194 images belonging to 2 classes.


## Building the model

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

In [6]:
INPUT = (150, 150, 3)

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu', input_shape=INPUT))
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
          
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9248)              0

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training the  model

In [9]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    train,
    epochs=10,
    validation_data=test,
    verbose=1,
    callbacks=[es]
)

Epoch 1/12
83/83 [==============================] - 30s 275ms/step - loss: 0.4533 - accuracy: 0.7823 - val_loss: 0.2337 - val_accuracy: 0.9278
Epoch 2/12
83/83 [==============================] - 20s 239ms/step - loss: 0.1750 - accuracy: 0.9452 - val_loss: 0.1352 - val_accuracy: 0.9485
Epoch 3/12
83/83 [==============================] - 20s 243ms/step - loss: 0.1625 - accuracy: 0.9429 - val_loss: 0.1041 - val_accuracy: 0.9639
Epoch 4/12
83/83 [==============================] - 19s 234ms/step - loss: 0.1063 - accuracy: 0.9703 - val_loss: 0.0922 - val_accuracy: 0.9536
Epoch 5/12
83/83 [==============================] - 23s 271ms/step - loss: 0.0980 - accuracy: 0.9711 - val_loss: 0.1031 - val_accuracy: 0.9485
Epoch 6/12
83/83 [==============================] - 32s 391ms/step - loss: 0.1407 - accuracy: 0.9566 - val_loss: 0.1492 - val_accuracy: 0.9381
Epoch 7/12
83/83 [==============================] - 32s 392ms/step - loss: 0.1413 - accuracy: 0.9574 - val_loss: 0.1218 - val_accuracy: 0.9536

In [10]:
model.evaluate(test)

13/13 [==============================] - 5s 331ms/step - loss: 0.1010 - accuracy: 0.9588


[0.10098671913146973, 0.9587628841400146]

In [11]:
model.save('face_model_CNN.h5')